In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0" 

import numpy as np
import tensorflow as tf
import pandas as pd
import pyarabic.araby as araby
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.initializers import TruncatedNormal
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy
import torch
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from transformers import Trainer, TrainingArguments
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from datasets import load_dataset, Dataset, concatenate_datasets
import torch
import torch.nn as nn
from transformers import AutoModel, TrainingArguments, Trainer, BertModel
from transformers.modeling_outputs import SequenceClassifierOutput
import numpy as np
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', 1000)


fname = 'results'
log_file = fname + '.txt'

with open(log_file, 'w') as f:
    f.write('Model,Accuracy,Precision,Recall,F1\n')


df = pd.read_csv('Expectation_Analysis.csv', encoding='utf-8', engine='python', sep='\t')



df.fillna('', inplace=True)  
display(len(df))


display(df.columns)
display(len(df))
display(df[:4])



classes = set(df['future_sentiment'].values)
display(classes)

c = df['future_sentiment'].value_counts()
display(c)

df['future_sentiment'] = df['future_sentiment'].astype('category')
df['label'] = df['future_sentiment'].cat.codes

df = df[['text', 'label']]
classes_num = len(classes)
display(classes_num)
display(len(df))


max_sequence_length = 128



models = [ 
        'aubmindlab/bert-base-arabertv02',
        'asafaya/bert-base-arabic',
        'CAMeL-Lab/bert-base-arabic-camelbert-da',
        'qarib/bert-base-qarib', 
        'UBC-NLP/ARBERTv2',
        'UBC-NLP/MARBERTv2',
        'faisalq/SaudiBERT',           
]


seeds = [0, 1, 42, 100]


for model_name in models:
    for seed in seeds:
        ds = Dataset.from_pandas(df)
        ds = ds.train_test_split(test_size=0.2, seed = seed)
        if seed==0:
            display(ds)
        for i in range(3):
            print(f'{model_name}, try:{i}')
                  
          
            tokenizer = AutoTokenizer.from_pretrained(model_name)                
            model = AutoModelForSequenceClassification.from_pretrained(model_name,
                                                                  num_labels=classes_num).to('cuda')                                                 
            dataset_train = ds['train']
            dataset_validation = ds['test']                                                    
            
          
    
            def preprocess_function(examples):
                return tokenizer(examples['text'], truncation=True, padding="max_length",
                                max_length=max_sequence_length)
            
            
            dataset_train = dataset_train.map(preprocess_function, batched=True) 
            dataset_validation = dataset_validation.map(preprocess_function, batched=True)  
            
           
            
            def compute_metrics(eval_pred):
                logits, labels = eval_pred
                predictions = np.argmax(logits, axis=-1)    
                acc = accuracy_score(labels, predictions)        
                f1 = f1_score(labels, predictions, average='macro')  
                precision = precision_score(labels, predictions, average='macro')
                recall = recall_score(labels, predictions, average='macro')
                with open(log_file, 'a') as f:
                    f.write(f'{model_name},{acc},{precision},{recall},{f1}\n')
                return {'accuracy': acc, 'precision': precision, 'recall':recall, 'f1_score': f1}
    
    
            
            
            epochs = 8
            save_steps = 10000 #save checkpoint every 10000 steps
            batch_size = 64
            
            training_args = TrainingArguments(
                output_dir = 'bert/',
                overwrite_output_dir=True,
                num_train_epochs = epochs,
                per_device_train_batch_size = batch_size,
                per_device_eval_batch_size = batch_size,
                save_steps = save_steps,
                save_total_limit = 1, #only save the last 5 checkpoints
                fp16=True,
                learning_rate = 5e-5,  # 5e-5 is the default
                logging_steps = 50, #50_000
                evaluation_strategy = 'steps',
                # evaluate_during_training = True,
                eval_steps = 50
                
            )
            
            trainer = Trainer(
                model = model,
                args = training_args,
                train_dataset=dataset_train,
                eval_dataset=dataset_validation,
                compute_metrics = compute_metrics
            )
            
            
            trainer.train()


results = pd.read_csv(log_file)

best_results = results.groupby('Model', as_index=False)['F1'].max()

best_results = pd.merge(best_results, results, on=['Model', 'F1'])
best_results = best_results[['Model', 'Accuracy', 'Precision', 'Recall', 'F1']]
best_results = best_results.drop_duplicates()
best_results.to_csv(f'{fname}.csv')
display(best_results)



2025-02-10 14:19:48.250489: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-10 14:19:48.412371: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-02-10 14:19:49.085251: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


16152

Index(['text', 'future_sentiment', 'class'], dtype='object')

16152

,text,future_sentiment,class
0,إن شاء الله خسارة للمتغطرس مدرب اليونايتد,pessimistic,sport
1,إن شاء الله عمان تغادر البطولة,optimistic,sport
2,اتمنى استغلال السمعة الاوروبية عن فيغا وتصريفه سريع في الشتوية,optimistic,sport
3,اتوقع انهيار الخصم الان,pessimistic,sport


...

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at faisalq/SaudiBERT and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/12921 [00:00<?, ? examples/s]

Map:   0%|          | 0/3231 [00:00<?, ? examples/s]

Step,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1 Score
50,0.530100,0.314365,0.869700,0.860837,0.871255,0.864643
100,0.329300,0.275205,0.886722,0.890797,0.879795,0.881330
150,0.271100,0.223442,0.909006,0.907274,0.901471,0.904148
200,0.251300,0.215061,0.916125,0.908409,0.918039,0.911272
250,0.109800,0.271255,0.909935,0.912896,0.899009,0.904714
300,0.107600,0.249120,0.915506,0.914811,0.908260,0.910975
350,0.105000,0.211077,0.921077,0.922745,0.912978,0.916706
400,0.111100,0.208813,0.915506,0.913170,0.912327,0.911795
450,0.043100,0.337737,0.906840,0.912449,0.891950,0.899293
500,0.058100,0.406554,0.895388,0.892301,0.894694,0.888127


,Model,Accuracy,Precision,Recall,F1
0,CAMeL-Lab/bert-base-arabic-camelbert-da,0.822037,0.812200,0.813821,0.812903
3,UBC-NLP/ARBERTv2,0.835655,0.828308,0.827383,0.827834
6,UBC-NLP/MARBERTv2,0.877128,0.872997,0.872043,0.872505
9,asafaya/bert-base-arabic,0.805633,0.796996,0.801571,0.798801
12,aubmindlab/bert-base-arabertv02,0.854225,0.846953,0.851188,0.848533
13,faisalq/SaudiBERT,0.933767,0.931596,0.930681,0.931105
16,qarib/bert-base-qarib,0.870319,0.863613,0.864173,0.863866
